In [ ]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import os
print(tf.__version__)

In [ ]:
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的图像默认为uint8（0-255的数字）。以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)   #[60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)   #[10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)  #[60000]
        self.test_label = self.test_label.astype(np.int32)   #[10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]
        self.train_dataset = tf.data.Dataset.from_tensor_slices((self.train_data, self.train_label))
        self.train_dataset = self.train_dataset.shuffle(buffer_size=23000)
        self.train_dataset = self.train_dataset.batch(batch_size)
        self.train_dataset = self.train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
        self.test_dataset = tf.data.Dataset.from_tensor_slices((self.test_data, self.test_label))
        self.test_dataset = self.test_dataset.shuffle(buffer_size=23000)
        self.test_dataset = self.test_dataset.batch(batch_size)
        self.test_dataset = self.test_dataset.prefetch(tf.data.experimental.AUTOTUNE)

class GoogleNet_BasicBlock(tf.keras.layers.Layer):
    def __init__(self, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool):
        super(GoogleNet_BasicBlock, self).__init__()
        
        self.conv_1x1 = self.conv2d(1, filters_1x1, 1)
        self.conv_3x3_reduce = self.conv2d(1, filters_3x3_reduce, 1)
        self.conv_3x3 = self.conv2d(3, filters_3x3, 1)
        self.conv_5x5_reduce = self.conv2d(1, filters_5x5_reduce, 1)
        self.conv_5x5 = self.conv2d(5, filters_5x5, 1)
        self.pool = tf.keras.layers.MaxPool2D(pool_size=[3, 3], strides=1, padding='same')
        self.conv_pool = self.conv2d(1, filters_pool, 1)
        self.concatenate = tf.keras.layers.Concatenate()

    def call(self, inputs, training=False):
        x1 = self.conv_1x1(inputs)
        x2 = self.conv_3x3_reduce(inputs)
        x2 = self.conv_3x3(x2)
        x3 = self.conv_5x5_reduce(inputs)
        x3 = self.conv_5x5(x3)
        x4 = self.pool(inputs)
        x4 = self.conv_pool(x4)
        output = self.concatenate([x1, x2, x3, x4])
        return output
    
    def conv2d(self, size, filters, stride):
        return tf.keras.layers.Conv2D(
            filters = filters,
            kernel_size = [size, size],
            strides = (stride, stride),
            padding = 'same',
            activation = tf.nn.relu,
            kernel_regularizer=tf.keras.regularizers.l2(l=val_l2),
            bias_regularizer=tf.keras.regularizers.l2(l=val_l2)
        )

class GoogleNet_with_block(tf.keras.Model):
    def __init__(self):
        super().__init__()

        self.inception1 = GoogleNet_BasicBlock(32, 16, 32, 8, 16, 16)
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.inception2 = GoogleNet_BasicBlock(32, 16, 32, 8, 16, 16)
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(units=10, activation=tf.nn.softmax)

    def call(self, inputs, training=False):
        out = self.inception1(inputs)          # (28, 28, 96)
        out = self.bn1(out, training=training)
        out = self.pool1(out)            # (14, 14, 96)
        
        out = self.inception2(out)             # (14, 14, 96)
        out = self.bn2(out, training=training)
        out = self.pool2(out)            # (7, 7, 96)
           
        out = self.flatten(out)                # (1, 1, 4704)
        out = self.dense(out)
        
        return out
    
    def conv2d(self, size, filters, stride):
        return tf.keras.layers.Conv2D(
            filters = filters,
            kernel_size = [size, size],
            strides = (stride, stride),
            padding = 'same',
            activation = tf.nn.relu
        )

class GoogleNet_without_block(tf.keras.Model):
    def __init__(self):
        super().__init__()

        self.conv_1x1_1 = self.conv2d(1, 32, 1)
        self.conv_3x3_reduce_1 = self.conv2d(1, 16, 1)
        self.conv_3x3_1 = self.conv2d(3, 32, 1)
        self.conv_5x5_reduce_1 = self.conv2d(1, 8, 1)
        self.conv_5x5_1 = self.conv2d(5, 16, 1)
        self.pool_1 = tf.keras.layers.MaxPool2D(pool_size=[3, 3], strides=1, padding='same')
        self.conv_pool_1 = self.conv2d(1, 16, 1)
        self.concatenate_1 = tf.keras.layers.Concatenate()
        
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        
        self.conv_1x1_2 = self.conv2d(1, 32, 1)
        self.conv_3x3_reduce_2 = self.conv2d(1, 16, 1)
        self.conv_3x3_2 = self.conv2d(3, 32, 1)
        self.conv_5x5_reduce_2 = self.conv2d(1, 8, 1)
        self.conv_5x5_2 = self.conv2d(5, 16, 1)
        self.pool_2 = tf.keras.layers.MaxPool2D(pool_size=[3, 3], strides=1, padding='same')
        self.conv_pool_2 = self.conv2d(1, 16, 1)
        self.concatenate_2 = tf.keras.layers.Concatenate()
        
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(units=10, activation=tf.nn.softmax)
        
        
    def call(self, inputs, training=False):
        #inception1
        x1 = self.conv_1x1_1(inputs)
        x2 = self.conv_3x3_reduce_1(inputs)
        x2 = self.conv_3x3_1(x2)
        x3 = self.conv_5x5_reduce_1(inputs)
        x3 = self.conv_5x5_1(x3)
        x4 = self.pool_1(inputs)
        x4 = self.conv_pool_1(x4)
        out = self.concatenate_1([x1, x2, x3, x4])
        
        out = self.bn1(out, training=training)
        out = self.pool1(out)            # (14, 14, 96)
        
        #inception2
        x1 = self.conv_1x1_2(out)
        x2 = self.conv_3x3_reduce_2(out)
        x2 = self.conv_3x3_2(x2)
        x3 = self.conv_5x5_reduce_2(out)
        x3 = self.conv_5x5_2(x3)
        x4 = self.pool_2(out)
        x4 = self.conv_pool_2(x4)
        out = self.concatenate_2([x1, x2, x3, x4])
        
        out = self.bn2(out, training=training)
        out = self.pool2(out)            # (7, 7, 96)
           
        out = self.flatten(out)                # (1, 1, 4704)
        out = self.dense(out)
        
        return out
    
    def conv2d(self, size, filters, stride):
        return tf.keras.layers.Conv2D(
            filters = filters,
            kernel_size = [size, size],
            strides = (stride, stride),
            padding = 'same',
            activation = tf.nn.relu
        )

num_epochs = 20
batch_size = 128
learning_rate = 0.001
val_l2 = 0.01

def train(train_dataset, test_dataset):
    history = model.fit(
         train_dataset,
         epochs=num_epochs,
         verbose=1,
         validation_data=test_dataset,
         callbacks=[cp_callback_mc]
    )
    return history
    
def train_generator(train_data, train_label, test_dataset):
    datagen = tf.keras.preprocessing.image.ImageDataGenerator()    
    datagen.fit(train_data)

    history = model.fit_generator(
         datagen.flow(train_data, train_label, batch_size=batch_size),
         steps_per_epoch=len(train_data) / batch_size,
         epochs=num_epochs,
         verbose=1,
         validation_data=test_dataset,
         callbacks=[cp_callback_mc]
         )
    return history

def test():
    loss_and_metrics = model.evaluate(test_dataset)
    print(loss_and_metrics)
    
if __name__ == '__main__':
    checkpoint_path = "block_speed_test_1/cp-{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)

    model = GoogleNet_with_block()
    model.build(input_shape=(None, 28, 28, 1))
    model.summary()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                    loss=tf.keras.losses.sparse_categorical_crossentropy,
                    metrics=[tf.keras.metrics.sparse_categorical_accuracy])
    
    data_loader = MNISTLoader()
    
    cp_callback_mc = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                        save_weights_only=True,
                                                        verbose=0)
    #latest = tf.train.latest_checkpoint(checkpoint_dir)
    #if(latest != None):
        #model.load_weights(latest) 
    

In [ ]:
    start = datetime.now()
    history = train(data_loader.train_dataset, data_loader.test_dataset)
    end = datetime.now()
    print(end - start)